##### Домашнее задание

## Урок №5 - Объединение таблиц. Представления. CTE

#### Работу выполнил: Зензеров Никита Евгеньевич

собрать датасет:
* ID аккаунта из account
* Количество персонажей
* Сумма транзакций по shop_log
* Количество походов в подземелья
* Количество походов в PvP

Избавляемся от NULL в выводе

дополнительно: за 7 дней до последнего входа аккаунта

In [2]:
# Сначала я выведу все таблицы, чтобы они были перед глазами. Так как я их не помню, воспользуюсь информационной схемой

In [3]:
info = pd.read_sql(f'''
SELECT *
FROM information_schema.tables
WHERE table_schema='{GROUP_SCHEMA}'
''', con=engine)
info

,table_catalog,table_schema,table_name,table_type,self_referencing_column_name,reference_generation,user_defined_type_catalog,user_defined_type_schema,user_defined_type_name,is_insertable_into,is_typed,commit_action
0,game-analytics,gu_game_analytics_1188,pvp_log_kozlov,BASE TABLE,None,None,None,None,None,YES,NO,None
1,game-analytics,gu_game_analytics_1188,shop_items,BASE TABLE,None,None,None,None,None,YES,NO,None
2,game-analytics,gu_game_analytics_1188,pvp_and_characters_log,VIEW,None,None,None,None,None,NO,NO,None
3,game-analytics,gu_game_analytics_1188,pvp_log,BASE TABLE,None,None,None,None,None,YES,NO,None
4,game-analytics,gu_game_analytics_1188,pvp_log_bokman,BASE TABLE,None,None,None,None,None,YES,NO,None
5,game-analytics,gu_game_analytics_1188,race_class,BASE TABLE,None,None,None,None,None,YES,NO,None
6,game-analytics,gu_game_analytics_1188,pvp_log_characters,VIEW,None,None,None,None,None,NO,NO,None
7,game-analytics,gu_game_analytics_1188,dungeon_log_makeev,BASE TABLE,None,None,None,None,None,YES,NO,None
8,game-analytics,gu_game_analytics_1188,charrace_count,VIEW,None,None,None,None,None,NO,NO,None
9,game-analytics,gu_game_analytics_1188,accounts,BASE TABLE,None,None,None,None,None,YES,NO,None


In [4]:
accounts = pd.read_sql(f'''
SELECT *
FROM {GROUP_SCHEMA}.accounts
LIMIT 10
''', con=engine)
accounts

,id,firstlogintime,lastlogintime
0,491878388,2017-06-30 02:32:05,2018-11-01 20:08:18
1,476978949,2017-01-05 19:34:51,2018-12-15 14:02:58
2,478705155,2016-12-30 20:30:21,2018-12-15 19:21:44
3,491534072,2017-06-23 17:51:25,2018-12-15 17:41:34
4,461343275,2017-04-27 22:50:24,2018-12-15 23:08:16
5,462097566,2017-05-24 21:34:02,2018-12-12 11:49:15
6,808719591,2018-03-23 02:48:27,2018-10-10 10:57:51
7,911413369,2018-08-23 23:19:03,2018-08-31 17:14:53
8,911421732,2018-08-24 02:15:35,2018-08-31 16:07:22
9,491077795,2017-06-16 14:28:06,2018-11-19 23:36:08


In [5]:
characters = pd.read_sql(f'''
SELECT *
FROM {GROUP_SCHEMA}.characters
LIMIT 10
''', con=engine)
characters

,id,accountid,charclass,charrace,charlevel
0,2889047,461209664,Warrior,Elf,44
1,4700139,461209664,Bard,Orc,3
2,2872086,461306388,Warrior,Dwarf,55
3,2865452,461306607,Bard,Orc,43
4,3833202,461306607,Mage,Dwarf,55
5,4264624,461306607,Bard,Human,43
6,4518814,461306607,Archer,Human,43
7,4853957,461306607,Assasin,Human,43
8,4867728,461306607,Mage,Human,43
9,4890468,461306607,Assasin,Human,43


In [6]:
dungeon_log = pd.read_sql(f'''
SELECT *
FROM {GROUP_SCHEMA}.dungeon_log
LIMIT 10
''', con=engine)
dungeon_log

,time,charid,charlevel,actiontype,actionname,result
0,2018-08-29 10:02:56.248,5681513,43,Dungeon,Тайник бесов,SUCCESS
1,2018-08-22 20:28:22.342,3508925,46,Dungeon,Тайник бесов,SUCCESS
2,2018-08-22 20:26:30.030,5822388,20,Dungeon,Тайник бесов,SUCCESS
3,2018-08-22 20:18:12.392,5822388,20,Dungeon,Тайник бесов,SUCCESS
4,2018-08-30 04:39:17.768,5882350,15,Dungeon,Тайник бесов,SUCCESS
5,2018-08-30 03:58:32.442,5882350,15,Dungeon,Тайник бесов,SUCCESS
6,2018-08-29 07:36:25.434,3314461,44,Dungeon,Тайник бесов,SUCCESS
7,2018-08-22 20:59:08.464,5363705,30,Dungeon,Тайник бесов,SUCCESS
8,2018-08-29 19:29:43.864,4541620,23,Dungeon,Тайник бесов,SUCCESS
9,2018-08-29 08:49:08.304,5291149,17,Dungeon,Тайник бесов,SUCCESS


In [7]:
shop_log = pd.read_sql(f'''
SELECT *
FROM {GROUP_SCHEMA}.shop_log
LIMIT 10
''', con=engine)
shop_log

,time,accountid,charid,itemname,itemprice,itemcount
0,2018-08-01 00:03:10.088,492169036,5706508,Малый набор для усиления,500,1
1,2018-08-01 00:09:41.799,909385150,5728727,Малый набор для усиления,500,1
2,2018-08-01 00:20:06.895,461499108,2867292,Премиум статус,1600,1
3,2018-08-01 00:31:06.911,903420362,5263550,Малый набор для усиления,500,1
4,2018-08-01 01:19:21.731,464491245,5738381,Малый набор для усиления,500,1
5,2018-08-01 01:20:46.129,490600378,5552754,Средний набор для усиления,950,2
6,2018-08-01 01:21:15.740,490278908,5707664,Малый набор для усиления,500,1
7,2018-08-01 01:30:05.782,490142864,5730766,Средний набор для усиления,950,1
8,2018-08-01 01:37:33.194,489392707,4857934,Премиум статус,1600,1
9,2018-08-01 01:37:54.196,489392707,4857934,Малый набор для усиления,500,2


In [8]:
pvp_log = pd.read_sql(f'''
SELECT *
FROM {GROUP_SCHEMA}.pvp_log
LIMIT 10
''', con=engine)
pvp_log

,time,charid,charlevel,actiondata
0,2018-08-28 16:36:48.105,5883488,43,"{""ActionType"":""PvP"",""ActionName"":""TeamDeathMat..."
1,2018-08-31 20:06:20.428,5064167,44,"{""ActionType"":""PvP"",""ActionName"":""QuickMatch,""..."
2,2018-08-28 18:09:02.843,5873839,17,"{""ActionType"":""PvP"",""ActionName"":""QuickMatch,""..."
3,2018-08-09 13:47:07.369,5822602,43,"{""ActionType"":""PvP"",""ActionName"":""QuickMatch,""..."
4,2018-08-01 22:30:29.094,3148382,54,"{""ActionType"":""PvP"",""ActionName"":""QuickMatch,""..."
5,2018-08-01 22:23:14.583,3148382,54,"{""ActionType"":""PvP"",""ActionName"":""QuickMatch,""..."
6,2018-08-19 22:28:09.986,5863397,14,"{""ActionType"":""PvP"",""ActionName"":""QuickMatch,""..."
7,2018-08-23 20:41:16.332,5873107,20,"{""ActionType"":""PvP"",""ActionName"":""QuickMatch,""..."
8,2018-08-09 18:32:05.838,5833009,16,"{""ActionType"":""PvP"",""ActionName"":""QuickMatch,""..."
9,2018-08-05 16:51:31.703,5661042,16,"{""ActionType"":""PvP"",""ActionName"":""QuickMatch,""..."


In [9]:
df = pd.read_sql(f'''
SELECT  a.id AS account_id,
        COUNT (DISTINCT c.id) AS char_count,
        COALESCE (trs, 0) AS transaction_sum,
        CAST (COALESCE (SUM (dg), 0) AS int) AS dungeon_log_count,
        CAST (COALESCE (SUM (pvp), 0) AS int) AS pvp_log_count
    FROM {GROUP_SCHEMA}.accounts AS a
    JOIN {GROUP_SCHEMA}.characters AS c ON a.id=c.accountid
    LEFT JOIN (
            SELECT accountid, SUM (itemprice * itemcount) AS trs 
            FROM {GROUP_SCHEMA}.shop_log 
            GROUP BY 1
        ) AS s ON a.id=s.accountid       
    LEFT JOIN (
            SELECT charid, COUNT (time) AS dg
            FROM {GROUP_SCHEMA}.dungeon_log
            GROUP BY 1
        ) AS d ON c.id=d.charid
    LEFT JOIN (
        SELECT charid, COUNT (time) AS pvp
        FROM {GROUP_SCHEMA}.pvp_log
        GROUP BY 1
        ) AS p ON c.id=p.charid
GROUP BY 1, 3
ORDER BY 3 DESC
''', con=engine)
df

,account_id,char_count,transaction_sum,dungeon_log_count,pvp_log_count
0,487589714,6,68800.0,141,0
1,492169036,8,67550.0,136,0
2,906428192,9,65750.0,135,1034
3,902023917,13,60300.0,265,217
4,462417041,2,49400.0,43,0
...,...,...,...,...,...
33606,911748920,3,0.0,3,0
33607,911765174,1,0.0,0,0
33608,911768039,1,0.0,0,0
33609,911776985,1,0.0,0,0


### Разница последнего логина и ивента может быть несколько месяцев, поэтому доп условие не применял

In [10]:
diff = pd.read_sql(f'''
SELECT MAX (CAST(lastlogintime AS date) - CAST(time AS date)) AS days_diff
FROM {GROUP_SCHEMA}.accounts AS a
    JOIN {GROUP_SCHEMA}.shop_log AS s ON a.id=s.accountid
LIMIT 10
''', con=engine)
diff

,days_diff
0,137


In [11]:
display(df.info())
print('Количество пропусков')
display(df.isnull().sum())
print('Количество дубликатов')
display(df.duplicated().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33611 entries, 0 to 33610
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   account_id         33611 non-null  int64  
 1   char_count         33611 non-null  int64  
 2   transaction_sum    33611 non-null  float64
 3   dungeon_log_count  33611 non-null  int64  
 4   pvp_log_count      33611 non-null  int64  
dtypes: float64(1), int64(4)
memory usage: 1.3 MB


None

Количество пропусков


account_id           0
char_count           0
transaction_sum      0
dungeon_log_count    0
pvp_log_count        0
dtype: int64

Количество дубликатов


0